In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta

# modeling & tuning
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# explainability
import shap
from lime.lime_tabular import LimeTabularExplainer
from pdpbox import pdp, info_plots

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# set path to your dataset csv
data_path = "high_dim_financial_dataset.csv"
df = pd.read_csv(data_path, parse_dates=['date'])
df = df.sort_values('date').reset_index(drop=True)
df.head()

print("shape:", df.shape)
print(df.columns.tolist())
print(df['date'].min(), df['date'].max())

# compute 5-day forward return and direction
df['return_5d'] = df['close'].pct_change(periods=5).shift(-5)  # forward 5-day return
df['target'] = (df['return_5d'] > 0).astype(int)  # 1 if price up in 5 days else 0

# drop rows where target is NaN (end of series)
df = df.dropna(subset=['target']).reset_index(drop=True)

def add_lag_rolling_features(df, cols, lags=[1,2,3,5,10], windows=[3,5,10,21]):
    for c in cols:
        for l in lags:
            df[f"{c}_lag{l}"] = df[c].shift(l)
        for w in windows:
            df[f"{c}_rollmean{w}"] = df[c].rolling(window=w).mean()
            df[f"{c}_rollstd{w}"] = df[c].rolling(window=w).std()
            df[f"{c}_rollmin{w}"] = df[c].rolling(window=w).min()
            df[f"{c}_rollmax{w}"] = df[c].rolling(window=w).max()
    return df

price_cols = ['close', 'volume']  # extend with other continuous columns you have
df = add_lag_rolling_features(df, price_cols, lags=[1,2,3,5,10], windows=[3,5,10,21])

# percent change features for stationarity (esp. macro series)
for c in price_cols:
    df[f"{c}_pct_1"] = df[c].pct_change(1)
    df[f"{c}_pct_5"] = df[c].pct_change(5)

df = df.dropna().reset_index(drop=True)
print("After feature engineering:", df.shape)

#Task 1 end
# exclude date, target, return_5d columns
exclude = ['date', 'target', 'return_5d']
feature_cols = [c for c in df.columns if c not in exclude]

# remove near-constant columns
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold=1e-5)
vt.fit(df[feature_cols])
cols_nonconst = np.array(feature_cols)[vt.get_support()].tolist()

print(f"Kept {len(cols_nonconst)} non-constant features from {len(feature_cols)}")
feature_cols = cols_nonconst

# correlation thresholding
corr = df[feature_cols].corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print("Dropping correlated:", len(to_drop))
feature_cols = [c for c in feature_cols if c not in to_drop]

#Task 2
# simple time-based split: train 70%, valid 15%, test 15%
n = len(df)
train_end = int(0.7 * n)
valid_end = int(0.85 * n)

train_df = df.iloc[:train_end].copy()
valid_df = df.iloc[train_end:valid_end].copy()
test_df  = df.iloc[valid_end:].copy()

X_train = train_df[feature_cols]
y_train = train_df['target']
X_valid = valid_df[feature_cols]
y_valid = valid_df['target']
X_test = test_df[feature_cols]
y_test = test_df['target']

print("Train/Valid/Test sizes:", X_train.shape, X_valid.shape, X_test.shape)

#Baseline: XGBoost pipeline + hyperparameter tuning (Task 2)
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbosity=0
)

pipe = Pipeline([
    ('scaler', StandardScaler()),   # scale continuous features
    ('model', xgb_clf)
])

param_grid = {
    'model__n_estimators': [100, 300],
    'model__max_depth': [3, 6],
    'model__learning_rate': [0.01, 0.1],
    'model__subsample': [0.8, 1.0]
}

gsearch = GridSearchCV(
    pipe,
    param_grid,
    cv=tscv,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

gsearch.fit(X_train, y_train)
print("Best params:", gsearch.best_params_)
print("Best CV AUC:", gsearch.best_score_)

#Evaluate on validation and test sets:
best = gsearch.best_estimator_
for name, X, y in [('VALID', X_valid, y_valid), ('TEST', X_test, y_test)]:
    preds_proba = best.predict_proba(X)[:,1]
    preds = best.predict(X)
    print(f"--- {name} ---")
    print("Accuracy:", accuracy_score(y, preds))
    print("ROC AUC:", roc_auc_score(y, preds_proba))
    print(classification_report(y, preds))
    print(confusion_matrix(y, preds))

#Model interpretability — SHAP (Task 3)
# prepare explainer - if using pipeline, extract the trained XGB model for SHAP
xgb_model = best.named_steps['model']
scaler = best.named_steps['scaler']

# We will explain standardized features (the model sees scaled features)
X_train_scaled = scaler.transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled  = scaler.transform(X_test)

# SHAP TreeExplainer
explainer = shap.TreeExplainer(xgb_model)
# compute shap_values on a sample to save time
sample_idx = np.random.choice(X_test_scaled.shape[0], size=min(2000, X_test_scaled.shape[0]), replace=False)
shap_values_raw = explainer.shap_values(X_test_scaled[sample_idx])

# Determine if shap_values_raw is a list (for classification) or a single array (sometimes for classification or regression)
if isinstance(shap_values_raw, list):
    # For binary classification, typically shap_values_raw is a list of two arrays: [shap_values_class_0, shap_values_class_1]
    shap_values_positive_class = shap_values_raw[1]
    expected_value_positive_class = explainer.expected_value[1]
else:
    # If it's a single array, assume it's for the positive class directly
    shap_values_positive_class = shap_values_raw
    expected_value_positive_class = explainer.expected_value # This should be a scalar in this case

# convert to DataFrame with original column names
shap_df = pd.DataFrame(shap_values_positive_class, columns=feature_cols)
shap_abs_mean = np.abs(shap_df).mean().sort_values(ascending=False)
top_shap = shap_abs_mean.head(30)
print("Top SHAP features:\n", top_shap.head(20))

shap.summary_plot(shap_values_positive_class, X_test_scaled[sample_idx], feature_names=feature_cols, plot_type="bar")

i_single_instance = 0  # choose the first sample from the sampled set for force plot
# For force plot, we need a single instance's SHAP values and expected value
shap.force_plot(
    expected_value_positive_class,
    shap_values_positive_class[i_single_instance],
    X_test_scaled[sample_idx][i_single_instance],
    feature_names=feature_cols,
    matplotlib=True
)
# OR for Jupyter interactive:
# shap.initjs(); shap.force_plot(expected_value_positive_class, shap_values_positive_class[i_single_instance], X_test_scaled[sample_idx][i_single_instance], feature_names=feature_cols)

# LIME needs the data in original (unscaled) or scaled space depending on how you fit your model.
# We'll create an explainer on scaled numpy arrays but we must pass a predict_proba wrapper.

X_train_np = X_train.values
X_test_np = X_test.values

# Create a wrapper to accept raw numpy -> apply scaler -> model predict_proba
def predict_proba_raw(X_raw):
    X_scaled = scaler.transform(X_raw)
    return xgb_model.predict_proba(X_scaled)

lime_explainer = LimeTabularExplainer(
    X_train_np,
    feature_names=feature_cols,
    class_names=['down','up'],
    discretize_continuous=True,
    random_state=RANDOM_STATE
)

# explain a test sample (raw)
idx = 10  # test sample index within X_test
exp = lime_explainer.explain_instance(X_test.iloc[idx].values, predict_proba_raw, num_features=10)
print(exp.as_list())
exp.show_in_notebook(show_table=True)

# pick top features from SHAP
import matplotlib.pyplot as plt
from pdpbox import pdp # Import the pdp module

top_features = list(top_shap.index[:5])

for feat in top_features:
    # Use pdp.pdp_isolate and pdp.pdp_plot
    try:
        pdp_go = pdp.pdp_isolate(model=xgb_model,
                                 dataset=pd.DataFrame(scaler.transform(X_test), columns=feature_cols),
                                 model_features=feature_cols,
                                 feature=feat)
        # Use pdp_plot directly after importing
        pdp.pdp_plot(pdp_go, feat)
        plt.show()
    except Exception as e:
        print("PDP failed for", feat, e)

#Generate three data-driven investment hypotheses (Task 4)
# 1) Identify top features and their mean SHAP sign (positive or negative impact)
shap_mean = shap_df.mean().sort_values(key=lambda s: np.abs(s), ascending=False)
top_feats = shap_mean.head(10)

hypotheses = []
for feat in top_feats.index[:6]:
    mean_shap = shap_df[feat].mean()
    direction = "increases" if mean_shap > 0 else "decreases"
    magnitude = mean_shap
    # compute change in model probability between 25th and 75th percentile of the feature using PDP
    feat_vals = pd.DataFrame(scaler.transform(X_test), columns=feature_cols)[feat]
    q25, q75 = np.percentile(feat_vals, [25,75])
    try:
        pdp_go = pdp.pdp_isolate(model=xgb_model,
                                 dataset=pd.DataFrame(scaler.transform(X_test), columns=feature_cols),
                                 model_features=feature_cols,
                                 feature=feat)
        # estimate expected probability change
        prob_q25 = np.interp(q25, pdp_go.feature_grids, pdp_go.pdp)
        prob_q75 = np.interp(q75, pdp_go.feature_grids, pdp_go.pdp)
        delta = prob_q75 - prob_q25
    except Exception:
        # fallback: use shap mean sign only
        delta = np.nan

    hypotheses.append({
        "feature": feat,
        "mean_shap": mean_shap,
        "direction": direction,
        "pdp_delta": delta
    })

hypotheses_df = pd.DataFrame(hypotheses)
hypotheses_df.head(6)


def mk_statement(row):
    feat = row['feature']
    dir_word = "higher" if row['mean_shap'] > 0 else "lower"
    pdp_text = (" estimated PDP probability change {:.4f}".format(row['pdp_delta'])
                if not np.isnan(row['pdp_delta']) else "")
    return f"Hypothesis: When {feat} is {dir_word}, the model predicts a higher chance of 5-day price increase — SHAP mean={row['mean_shap']:.4f}.{pdp_text}"

for r in hypotheses_df.head(3).to_dict('records'):
    print(mk_statement(r))


# Reinstall pdpbox to resolve potential installation issues
!pip uninstall pdpbox -y
!pip install pdpbox

report_lines = []
report_lines.append("Project: Interpretable ML for High-Dimensional Financial Time Series\n")
report_lines.append("Methodology:\n1) Feature engineering: lags, rolling stats, pct changes\n2) Model: XGBoost time-series-aware CV and grid search\n3) Interpretability: SHAP, LIME, PDP\n")
report_lines.append("\nModel performance (VALID):\n")
# insert printed metrics
val_preds = best.predict(X_valid)
val_proba  = best.predict_proba(X_valid)[:,1]
report_lines.append(classification_report(y_valid, val_preds))
report_lines.append("\nTop SHAP features:\n")
report_lines.extend([f"{f}: {v:.6f}\n" for f,v in top_shap.items()])

# add hypotheses text
report_lines.append("\nInvestment Hypotheses:\n")
for r in hypotheses_df.head(3).to_dict('records'):
    report_lines.append(mk_statement(r) + "\n")

with open("submission_report.txt", "w") as f:
    f.writelines(report_lines)
print("Saved submission_report.txt and saved model at models/xgb_ts_best.pkl")

plt.figure(figsize=(8,4))
top_shap.head(20).plot.barh()
plt.title("Top SHAP mean(|value|)")
plt.tight_layout()
plt.savefig("top_shap_bar.png")

